# 定数設定

In [42]:
%cd /content


/content


In [0]:
%rm -rf /content/attention-analysis

In [45]:
!pip install sentencepiece

In [46]:
!git clone https://github.com/HisakaKoji/attention-analysis.git

Cloning into 'attention-analysis'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 83 (delta 32), reused 57 (delta 16), pack-reused 0
Unpacking objects: 100% (83/83), done.


fatal: not a git repository (or any of the parent directories): .git


In [48]:
%cd attention-analysis

/content/attention-analysis


In [73]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/HisakaKoji/attention-analysis
   2c46b65..05e83bb  master     -> origin/master
Updating 2c46b65..05e83bb
Fast-forward
 attns.py | 2 ++
 1 file changed, 2 insertions(+)


In [0]:
from google.colab import auth
auth.authenticate_user()

In [51]:
!gsutil cp gs://hisaka/model/model.ckpt-1400000*   ./model/bert-wiki-ja/

Copying gs://hisaka/model/model.ckpt-1400000.data-00000-of-00001...
Copying gs://hisaka/model/model.ckpt-1400000.index...
Copying gs://hisaka/model/model.ckpt-1400000.meta...
\ [3 files][  1.3 GiB/  1.3 GiB]   71.3 MiB/s                                   
Operation completed over 3 objects/1.3 GiB.                                      


In [52]:
!gsutil cp gs://hisaka/model/wiki-ja.model  ./model/bert-wiki-ja/
!gsutil cp gs://hisaka/model/wiki-ja.vocab  ./model/bert-wiki-ja/

Copying gs://hisaka/model/wiki-ja.model...
/ [1 files][786.8 KiB/786.8 KiB]                                                
Operation completed over 1 objects/786.8 KiB.                                    
Copying gs://hisaka/model/wiki-ja.vocab...
/ [1 files][581.7 KiB/581.7 KiB]                                                
Operation completed over 1 objects/581.7 KiB.                                    


In [0]:
# デフォルトのサンプル以外のデータを処理したい場合は、dataフォルダにjsonファイルを置き、下記２行をjsonファイルの名前に変更
JSONFILE_PATH = './data/default_sample.json'
HTMLFILE_PATH = './data/default_sample.html'

In [0]:
# 出典: https://ja.wikipedia.org/wiki/%E3%83%AD%E3%83%9C%E3%83%83%E3%83%88%E5%B7%A5%E5%AD%A6%E4%B8%89%E5%8E%9F%E5%89%87
DEFAULT_SAMPLE = [
    { "class": "test", "info1": "ロボット三原則", "info2": "第一条", "text": "ロボットは人間に危害を加えてはならない。また、その危険を看過することによって、人間に危害を及ぼしてはならない。" },
    { "class": "test", "info1": "ロボット三原則", "info2": "第二条", "text": "ロボットは人間にあたえられた命令に服従しなければならない。ただし、あたえられた命令が、第一条に反する場合は、この限りでない。" },
    { "class": "test", "info1": "ロボット三原則", "info2": "第三条", "text": "ロボットは、前掲第一条および第二条に反するおそれのないかぎり、自己をまもらなければならない。" },
    ]

# 初期化

In [0]:
import codecs
import json
import os
from IPython.display import HTML

from attns import Attns

In [0]:
pklfile_path = JSONFILE_PATH.replace('.json', '_attn.pkl')
os.environ['JSONFILE_PATH'] = JSONFILE_PATH
os.environ['PKLFILE_PATH'] = pklfile_path

In [0]:
# デフォルト・サンプルの場合は、jsonファイルを保存
if JSONFILE_PATH == './data/default_sample.json':
    with codecs.open(JSONFILE_PATH, 'w', 'utf-8') as f:
        json.dump(DEFAULT_SAMPLE, f)

# アテンション抽出

In [83]:
!python3 extract_attention.py \
  --preprocessed-data-file $JSONFILE_PATH \
  --bert-dir ./model/bert-wiki-ja

Creating examples...
Loaded a trained SentencePiece model.

Building BERT model...




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.

Loading BERT from checkpoint...

Extracting attention maps...
2019-09-12 02:02:17.811971: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-09-12 02:02:17.812183: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2028840 executing computations on platform Host. Devices:
2019-09-12 02:02:17.812217: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-09-12 02:02:18.143850: W 

# 結果(HTML)保存

In [0]:
html_string = Attns(pklfile_path).to_html()

In [0]:
with codecs.open(HTMLFILE_PATH, 'w', 'utf-8') as f:
    f.write(html_string)

# 結果(HTML)表示　※先頭512KB

In [86]:
HTML(html_string[0:min(len(html_string), 512*1024)])

Head0,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head1,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head2,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head3,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head4,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head5,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head6,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head7,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head8,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head9,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
Head10,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...,&nbs...
